In [1]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation,Flatten
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras import metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle

from keras import metrics
from keras.layers.merge import concatenate
import sys
sys.path.append('../')
from Utilities.model_visualization import model_to_png

from PIL import Image # used for loading images
import numpy as np
from numpy import asarray
import matplotlib.pyplot as plt

import os # used for navigating to image path

from keras.layers import Input

import cv2
from keras.layers import concatenate
from keras.models import Model
from keras.applications import VGG16

import pandas as pd
from glob import glob
import tensorflow as tf

from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import StratifiedKFold
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras.callbacks import EarlyStopping

import pickle
from scipy import misc
from keras import optimizers

from keras.utils import plot_model

from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

import time

%matplotlib inline

Using TensorFlow backend.


<h1>Loading Data</h1>

In [2]:
start = time.time()
df = pd.read_pickle('dataset.pkl')
print(df.iloc[0])

text          HARVEY AFTER DONNA KISSED HIM: https://t.co/mz...
text_info                                       not_informative
image_path    data_image/hurricane_harvey/14_9_2017/90825435...
damage                                               irrelevant
Name: 0, dtype: object


In [3]:
split_pos = int(df['text'].count()*0.8)
train = df[:split_pos]
test = df[split_pos:]

In [4]:
def get_vgg_model():    # Pre Trained Embeddings
    
    #image classifier
    IMAGE_SIZE = [224, 224]  # we will keep the image size as (64,64). You can increase the size for better results. 
    vgg = VGG16(input_shape = (224, 224, 3), weights = None, include_top = True)  # input_shape = (64,64,3) as required by VGG
    x = (vgg.layers[-2].output)
    image_model = Dense(4, activation = 'softmax',name='Hybrid_Classifier')(x)  # adding the output layer with softmax function as this is a multi label classification problem.
    model = Model(inputs=[vgg.input], outputs=[image_model])
    return model

In [ ]:
filepath = "text_weights.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max', period=1)
early_stopping = EarlyStopping(monitor='val_acc', min_delta=0, patience=4, verbose=1, mode='max')
callbacks_list = [checkpoint, early_stopping]

In [ ]:
vgg_model = get_vgg_model()
vgg_model.compile(loss=['categorical_crossentropy'],
                       optimizer= optimizers.Nadam(lr=0.00075,
                                                   beta_1=0.5,
                                                   beta_2=0.999,
                                                   epsilon=None, 
                                                   schedule_decay=0.0001),
                       metrics=['accuracy',metrics.mae, metrics.categorical_accuracy])

<h1>CNN Image</h1>

In [ ]:
IMG_SIZE =224
dataset_dir = 'H:/FYP DATASETS/FYP DATASETS/Crisis/'

def load_img(img):
    path = os.path.join(dataset_dir, img)
    rows=224
    columns=224
    img= cv2.resize(cv2.imread(path,cv2.IMREAD_COLOR),(rows,columns),interpolation=cv2.INTER_CUBIC)
    return img

In [ ]:
for index, row in train.iterrows():
    train.at[index,'image_path'] = load_img(row['image_path'])

In [ ]:
def encode_label(damage):
    # integer encode
    damage = np.array(damage)
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(damage)
    # binary encode
    onehot_encoder = OneHotEncoder(sparse=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
    return onehot_encoded

In [ ]:
y = encode_label(train.iloc[:]['damage'])
print(train.iloc[0])

In [ ]:
print(train.damage.unique())

In [ ]:
train_images = train['image_path'].tolist()
# no need to convert y to list as it is 1 dim encoding takes care of it
train_images = np.array(train_images)
train_text = np.array(train['text'].tolist())

In [ ]:
filepath = "divided_checkpoints_1.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_vgg_output_acc', verbose=1, save_best_only=True, mode='max', period=1)
early_stopping = EarlyStopping(monitor='val_vgg_output_acc', min_delta=0, patience=6, verbose=1, mode='max')
callbacks_list = [checkpoint, early_stopping]
load_time = time.time() - start

In [ ]:
start = time.time()
vgg_history = vgg_model.fit(x=[train_images], y=[y],
                           epochs=60,
                           batch_size=10,
                           validation_split=0.25,
                           shuffle=True,
                           callbacks = callbacks_list,
                           verbose=1)
vgg_time = time.time() - start


In [ ]:
# start = time.time()

# vgg_history = vgg_model.fit(x=[train_images], y=[y],
#                            epochs=40,
#                            batch_size=10,
#                            validation_split=0.2,
#                            shuffle=True,
#                            verbose=1)
# vgg_time = time.time() - start

In [ ]:
print('vgg train time ' + str(vgg_time))
print('load time ' + str(load_time))

In [ ]:
hybrid_model.save_weights('vgg.h5')
#vgg_model.save_weights('vgg.h5')